# TDIDF

In this assignment, you will enhance a suboptimal information retrieval system. Your task is to develop an inverted index to facilitate the rapid retrieval of documents matching specific queries. Further improve your system by implementing term-frequency inverse-document-frequency (TF-IDF) weighting and using cosine similarity for comparing queries with your dataset. The performance of your information retrieval system will be assessed based on the accuracy of the document retrieval, as well as the precision of the computed TF-IDF values and cosine similarities.


## Data Description
For this assignment, you will use your Information Retrieval (IR) system to locate relevant documents within a collection of sixty books sourced from Project Gutenberg. The training data can be found in the `data/` directory, specifically under the `ProjectGutenberg/` subdirectory. Here, you will find another directory named `raw/`, which contains the raw text files for the sixty short stories. Additionally, the `data/` directory includes files `dev_queries.txt` and `dev_solutions.txt`, which contain a set of development queries and their corresponding expected answers. These resources are provided to aid you as you begin to implement your IR system.

## Goal

Enhance the existing IR system by developing the following functionalities:

1. Inverted Positional Index: Create an inverted index, which is a mapping of words to the documents they appear in, along with the positions within those documents where the words occur.

1. Boolean Retrieval: Implement a Boolean retrieval system that returns a list of documents containing all the words in a given query. For this assignment, your system should only need to support conjunctions.

1. TF-IDF: Calculate and store the term-frequency inverse-document-frequency (TF-IDF) values for every word-document pairing.

1. Cosine Similarity: To enhance the accuracy and relevance of document retrieval, you will implement cosine similarity. Here's a breakdown of how to apply cosine similarity with specific weighting schemes for both queries and documents:


#### Cosine Similarity

When computing the query term weight you will use $tf_{t,q} = 1+log_{10}(\text{count}(t,q))$ with no IDF term and no normalization.
When computing the document vector you will calculate the tf, idf and normalize it (divide by the square root of the sum of squares of the tf-idf weights) as seen in class. To calculate the cosine scores you will perform a dot product between the document vector and the query vector.

In [1]:
import os
import re
import sys
import json
import math
import collections
import numpy as np
from pprint import pprint  
from stemmer import PorterStemmer

In this exercise, you will only work on the `IRSystem` Class.

In the next cells, you will extend the functionality of this class. For better readability and ease of use, you will add additional methods to the same class by inheriting from itself. Use this example for reference:

```
class Foo:
    def a(self):
        print("a")

class Foo(Foo):
    def b(self):
        print("b")

foo = Foo()
foo.a()
foo.b()
```
The following will be printed without errors: 
```
a
b
```
The first part of the class will load and stem the data.

In [2]:
class IRSystem:
    def __init__(self):
        # For holding the data - initialized in read_data()
        self.titles = []
        self.docs = []
        self.vocab = []
        # For the text pre-processing.
        self.alphanum = re.compile('[^a-zA-Z0-9]')
        self.p = PorterStemmer()

    def get_uniq_words(self):
        uniq = set()
        for doc in self.docs:
            for word in doc:
                uniq.add(word)
        return uniq

    def __read_raw_data(self, dirname):
        print("Stemming Documents...")
        titles = []
        docs = []
        os.mkdir('%s/stemmed' % dirname)
        title_pattern = re.compile(r'(.*) \d+\.txt')
        
        # make sure we're only getting the files we actually want
        filenames = []
        for filename in os.listdir('%s/raw' % dirname):
            if filename.endswith(".txt") and not filename.startswith("."):
                filenames.append(filename)

        for i, filename in enumerate(filenames):
            title = title_pattern.search(filename).group(1)
            print("    Doc %d of %d: %s" % (i + 1, len(filenames), title))
            titles.append(title)
            contents = []
            f = open('%s/raw/%s' % (dirname, filename), 'r', encoding="utf-8")
            of = open('%s/stemmed/%s.txt' % (dirname, title), 'w',
                      encoding="utf-8")
            for line in f:
                # make sure everything is lower case
                line = line.lower()
                # split on whitespace
                line = [xx.strip() for xx in line.split()]
                # remove non alphanumeric characters
                line = [self.alphanum.sub('', xx) for xx in line]
                # remove any words that are now empty
                line = [xx for xx in line if xx != '']
                # stem words
                line = [self.p.stem(xx) for xx in line]
                # add to the document's conents
                contents.extend(line)
                if len(line) > 0:
                    of.write(" ".join(line))
                    of.write('\n')
            f.close()
            of.close()
            docs.append(contents)
        return titles, docs

    def __read_stemmed_data(self, dirname):
        print("Already stemmed!")
        titles = []
        docs = []

        # make sure we're only getting the files we actually want
        filenames = []
        for filename in os.listdir('%s/stemmed' % dirname):
            if filename.endswith(".txt") and not filename.startswith("."):
                filenames.append(filename)

        if len(filenames) != 60:
            msg = "There are no 60 documents in ../data/ProjectGutenberg/stemmed/\n"
            msg += "Remove ../data/ProjectGutenberg/stemmed/ directory and re-run."
            raise Exception(msg)

        for i, filename in enumerate(filenames):
            title = filename.split('.')[0]
            titles.append(title)
            contents = []
            f = open('%s/stemmed/%s' % (dirname, filename), 'r',
                     encoding="utf-8")
            for line in f:
                # split on whitespace
                line = [xx.strip() for xx in line.split()]
                # add to the document's conents
                contents.extend(line)
            f.close()
            docs.append(contents)

        return titles, docs

    def read_data(self, dirname):
        """
        Given the location of the 'data' directory, reads in the documents to
        be indexed.
        """
        # NOTE: We cache stemmed documents for speed
        #       (i.e. write to files in new 'stemmed/' dir).

        print("Reading in documents...")
        # dict mapping file names to list of "words" (tokens)
        filenames = os.listdir(dirname)
        subdirs = os.listdir(dirname)
        if 'stemmed' in subdirs:
            titles, docs = self.__read_stemmed_data(dirname)
        else:
            titles, docs = self.__read_raw_data(dirname)

        # Sort document alphabetically by title to ensure we have the proper
        # document indices when referring to them.
        ordering = [idx for idx, title in sorted(enumerate(titles),
                                                 key=lambda xx: xx[1])]

        self.titles = []
        self.docs = []
        numdocs = len(docs)
        for d in range(numdocs):
            self.titles.append(titles[ordering[d]])
            self.docs.append(docs[ordering[d]])

        # Get the vocabulary.
        self.vocab = [xx for xx in self.get_uniq_words()]

You will first build the inverted positional index. The documents will have already been read at this point. The following instance variables in the class are included in the starter code for you to use to build your inverted positional index: 
* titles (a list of strings)
* docs (a list of lists of strings)
* vocab (a list of strings).

Since the majority of the work in this assignment will use document ID, we recommend doing the mapping using the document IDs (i.e. the index of the document within self.docs). You can then retrieve the text and title of any given document by indexing into self.docs and self.titles respectively.

In [3]:
class IRSystem(IRSystem):
    def index(self):
        """
        Build an inverted positional index of the documents.
        """
        print("Indexing...")
        inv_index = {}

        #iterate over each document and its index
        for doc_id, doc in enumerate(self.docs):
            
            #iterate over each word in the document and its index
            for pos, word in enumerate(doc):
                if word not in inv_index:
                    inv_index[word] = {}
                if doc_id not in inv_index[word]:
                    inv_index[word][doc_id] = []
                inv_index[word][doc_id].append(pos)

        self.inv_index = inv_index

    def print_index(self):
        """
        A helper function to print the inverted index.
        This function is useful for debugging and visualizing the index.
        """
        pprint(self.inv_index)

Next we will implement `get_word_positions`. This method returns a list of integers that identifies the positions in the document doc (represented as document ID) in which the word is found. This is basically just an API into your inverted index, but you must implement it in order for the index to be evaluated fully.

Be careful when accessing your inverted index! Trying to index into a dictionary using a key that is not present will cause an error.

In [4]:
class IRSystem(IRSystem):
    def get_word_positions(self, word, doc):
        """
        Given a word and a document, use the inverted index to return
        the positions of the specified word in the specified document.
        """
        if word in self.inv_index and doc in self.inv_index[word]:
            return self.inv_index[word][doc]
        return []

We will add another method, `get_posting`, that returns a list of integers (document IDs) that identifies the documents in which the word is found. This is basically another API into your inverted index, but you must implement it in order to be evaluated fully.

In [5]:
class IRSystem(IRSystem):
    def get_posting(self, word):
        """
        Given a word, this returns the list of document indices (sorted) in
        which the word occurs.
        """
        if word in self.inv_index:
            return list(self.inv_index[word].keys())
        return []

    
    def get_posting_unstemmed(self, word):
        """
        Given a word, this *stems* the word and then calls get_posting on the
        stemmed word to get its postings list. You should *not* need to change
        this function. It is needed for submission.
        """
        word = self.p.stem(word)
        return self.get_posting(word)

Next, we will implement Boolean retrieval, returning a list of document IDs corresponding to the documents in which all the words in the query occur.

In [6]:
class IRSystem(IRSystem):
    def boolean_retrieve(self, query):
        """
        Given a query in the form of a list of *stemmed* words, this returns
        the list of documents in which *all* of those words occur (ie an AND
        query).
        Return an empty list if the query does not return any documents.
        """
         #check if the query is empty, if so - returns empty list
        if not query:
            return []

        #im using 'set' because its return unique values
        #'get' is used if the term in the query is not found it will return empty list
        docs = set(self.inv_index.get(query[0], []))

        #from the second term for every other term in the query i check if the term is in the inverted index.
        for term in query[1:]:
            if term in self.inv_index:
                docs &= set(self.inv_index[term])
            else:
                return []

        return sorted(docs)

Now we will compute and score the tf-idf values. `compute_tfidf` stores the tf-idf values for words and documents. For this, you will probably want to be aware of the class variable vocab, which holds the list of all unique words, as well as the inverted index you created earlier.

You must also implement get_tfidf to return the tf-idf weight for a particular word and document ID. Make sure you correctly handle the case where the word isn't present in your vocabulary.

In [7]:
class IRSystem(IRSystem):
    def compute_tfidf(self):
        print("Calculating tf-idf...")
        N = len(self.docs)  #total number of documents
        self.tfidf = {}  #create tf-idf dictionary

        for word in self.vocab:
            df_t = len(self.get_posting(word))  #df_t - number of documents that t appears in
            idf_t = math.log10(N / df_t) if df_t > 0 else 0  #avoid division by zero

            for doc_id in self.get_posting(word):
                tf_t_d = len(self.get_word_positions(word, doc_id))  #tf_t_d - number of t appears in d
                tf_idf_t_d = (1 + math.log10(tf_t_d)) * idf_t if tf_t_d > 0 else 0  #avoid log(0)
                
                if doc_id not in self.tfidf:
                    self.tfidf[doc_id] = {}  
                
                self.tfidf[doc_id][word] = tf_idf_t_d  #store the tf-idf value for the word in the document

    def get_tfidf(self, word, document):
        tfidf = 0.0
        if word in self.vocab and document in self.tfidf and word in self.tfidf[document]:
            tfidf = self.tfidf[document][word]
        return tfidf

    def get_tfidf_unstemmed(self, word, document):
        """
        This function gets the TF-IDF of an *unstemmed* word in a document.
        Stems the word and then calls get_tfidf. You should *not* need to
        change this interface, but it is necessary for submission.
        """
        word = self.p.stem(word)
        return self.get_tfidf(word, document)

Lastly, we will implement `rank_retrieve`. This function returns sorted list of the top ranked documents for a given query.

Remember to use ltc.lnn weighting, that is, ltc weighting for the document and lnn weighting for the query! This means that the query vector weights will be $tf_{t,q} = 1+log_{10}(\text{count}(t,q))$ with no IDF term or normalization (or 0 if the term is not present in the query), but we do normalize the document vector weights by the magnitude of the document vector (square root of the sum of squares of the tf-idf weights). Finally, the cosine scores are the dot product of the query vector and the document vectors.

In [8]:
class IRSystem(IRSystem):
    def rank_retrieve(self, query):
        """
        Given a query (a list of words), return a rank-ordered list of
        documents (by ID) and score for the query.
        """
        scores = [0.0 for xx in range(len(self.titles))]
        
        query_weights = {}
        for term in query:
            query_weights[term] = 1 + math.log10(query.count(term)) 
        
        
        doc_lengths = [0.0 for xx in range(len(self.titles))]
        for doc_id, doc in enumerate(self.docs):
                length = 0.0
                for word in set(doc):
                    length += self.get_tfidf(word, doc_id) ** 2 #sums up the square of the tf-idf weight of each word in the doc
                doc_lengths[doc_id] = math.sqrt(length) #square root of the sum in order to get the length of the vector |d|
        
        
        for term in query_weights:
            postings = self.get_posting(term) #find all the documents contain t in the query
            for doc_id in postings:
                scores[doc_id] += self.get_tfidf(term, doc_id) * query_weights[term] #numerator of the cosine similarity 

                
        for doc_id in range(len(scores)):
            if doc_lengths[doc_id] > 0:
                scores[doc_id] /= doc_lengths[doc_id] #normalize the scores

                
        ranking = [idx for idx, sim in sorted(enumerate(scores), key=lambda xx: xx[1], reverse=True)]
        results = []
        for i in range(10): #return only the top 10 documents related to the query and their tfidf score
            results.append((ranking[i], scores[ranking[i]]))
        return results

Let's also add a few methods that given a string, will process and then return the list of matching documents for the different methods you have implemented. 

You do not need to add any additional code here.

In [9]:
class IRSystem(IRSystem): 
    def process_query(self, query_str):
        """
        Given a query string, process it and return the list of lowercase,
        alphanumeric, stemmed words in the string.
        """
        # make sure everything is lower case
        query = query_str.lower()
        # split on whitespace
        query = query.split()
        # remove non alphanumeric characters
        query = [self.alphanum.sub('', xx) for xx in query]
        # stem words
        query = [self.p.stem(xx) for xx in query]
        return query

    def query_retrieve(self, query_str):
        """
        Given a string, process and then return the list of matching documents
        found by boolean_retrieve().
        """
        query = self.process_query(query_str)
        return self.boolean_retrieve(query)

    def phrase_query_retrieve(self, query_str):
        """
        Given a string, process and then return the list of matching documents
        found by phrase_retrieve().
        """
        query = self.process_query(query_str)
        return self.phrase_retrieve(query)

    def query_rank(self, query_str):
        """
        Given a string, process and then return the list of the top matching
        documents, rank-ordered.
        """
        query = self.process_query(query_str)
        return self.rank_retrieve(query)


## Running the code
You can use the `run_tests` and `run_query` functions to test your code. `run_tests` tests how different components your search engine code perform on a small set of queries and checks whether or not it matches up with the solution's results. `run_query` can be used to test your code on individual queries.

Note that the first run for either of these functions might take a little while, since it will stem all the words in every document create a directory named `stemmed/` in `../data/ProjectGutenberg/`. This is meant to be a simple cache for the stemmed versions of the text documents. Later runs will be much faster after the first run since all the stemming will already be completed. However, this means that if something happens during this first run and it does not get through processing all the documents, you may be left with an incomplete set of documents in `../data/ProjectGutenberg/stemmed/`. If this happens, simply remove the `stemmed/` directory and re-run.



In [10]:
def run_tests(irsys):
    print("===== Running tests =====")

    ff = open('./data/dev_queries.txt')
    questions = [xx.strip() for xx in ff.readlines()]
    ff.close()
    ff = open('./data/dev_solutions.txt')
    solutions = [xx.strip() for xx in ff.readlines()]
    ff.close()

    epsilon = 1e-4
    for part in range(5):
        points = 0
        num_correct = 0
        num_total = 0

        prob = questions[part]
        soln = json.loads(solutions[part])
        

        if part == 0:  # inverted index test
            print("Inverted Index Test")
            queries = prob.split("; ")
            queries = [xx.split(", ") for xx in queries]
            queries = [(xx[0], int(xx[1])) for xx in queries]
            for i, (word, doc) in enumerate(queries):
                num_total += 1
                guess = irsys.get_word_positions(word, doc)
                if sorted(guess) == soln[i]:
                    num_correct += 1

        if part == 1:  # get postings test
            print("Get Postings Test")
            words = prob.split(", ")
            for i, word in enumerate(words):
                num_total += 1
                posting = irsys.get_posting_unstemmed(word)
                if posting == soln[i]:
                    num_correct += 1

        elif part == 2:  # boolean retrieval test
            print("Boolean Retrieval Test")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                guess = irsys.query_retrieve(query)
                if set(guess) == set(soln[i]):
                    num_correct += 1

        elif part == 3:  # tfidf test
            print("TF-IDF Test")
            queries = prob.split("; ")
            queries = [xx.split(", ") for xx in queries]
            queries = [(xx[0], int(xx[1])) for xx in queries]
            for i, (word, doc) in enumerate(queries):
                num_total += 1
                guess = irsys.get_tfidf_unstemmed(word, doc)
                if guess >= float(soln[i]) - epsilon and \
                        guess <= float(soln[i]) + epsilon:
                    num_correct += 1

        elif part == 4:  # cosine similarity test
            print("Cosine Similarity Test")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                ranked = irsys.query_rank(query)
                top_rank = ranked[0]
                if top_rank[0] == soln[i][0]:
                    if top_rank[1] >= float(soln[i][1]) - epsilon and \
                            top_rank[1] <= float(soln[i][1]) + epsilon:
                        num_correct += 1

        feedback = "%d/%d Correct. Accuracy: %f" % \
                   (num_correct, num_total, float(num_correct) / num_total)
        print(feedback)

In [11]:
## Run this cell to run the tests
irsys = IRSystem()
irsys.read_data('./data/ProjectGutenberg')
irsys.index()
irsys.compute_tfidf()

run_tests(irsys)

Reading in documents...
Already stemmed!
Indexing...
Calculating tf-idf...
===== Running tests =====
Inverted Index Test
6/6 Correct. Accuracy: 1.000000
Get Postings Test
6/6 Correct. Accuracy: 1.000000
Boolean Retrieval Test
6/6 Correct. Accuracy: 1.000000
TF-IDF Test
6/6 Correct. Accuracy: 1.000000
Cosine Similarity Test
6/6 Correct. Accuracy: 1.000000


In [12]:
def run_query(query):
    irsys = IRSystem()
    irsys.read_data('./data/ProjectGutenberg')
    irsys.index()
    irsys.compute_tfidf()
    
    print("Best matching documents to '%s':" % query)
    results = irsys.query_rank(query)
    for docId, score in results:
        print("%s: %e" % (irsys.titles[docId], score))
        
# Run any query you want!
run_query("My very own query")

Reading in documents...
Already stemmed!
Indexing...
Calculating tf-idf...
Best matching documents to 'My very own query':
Anna Karenina - Leo Tolstoy: 1.477895e-02
The Scarlet Letter - Nathaniel Hawthorne: 1.324621e-02
Around the World in Eighty Days - Jules Verne: 1.291871e-02
Dracula - Bram Stoker: 1.134571e-02
Violin Mastery_Talks with Master Violinists and Teachers - Frederick Herman Martens: 1.129498e-02
The Iliad - Homer: 9.175461e-03
War and Peace - Leo Tolstoy: 6.636483e-03
Don Quixote - Miguel de Cervantes Saavedra: 6.257655e-03
The Count of Monte Cristo - Alexandre Dumas and Auguste Maquet: 5.863665e-03
Our Vanishing Wild Life_Its Extermination and Preservation - William T Hornaday: 5.861535e-03


For reference, you can run the cell below to see the titles of all the documents in our dataset. As sanity checks, you can try tailoring your queries in run_query to output certain titles and/or checking what IR system outputs against the list of titles to see if the results make sense (i.e. the book Great Pianists on Piano Playing should probably be among the top results if the query is "pianists play piano").

In [13]:
run_query("pianists play piano") #the book 'Great Pianists on Piano Playing' should probably be among the top results

Reading in documents...
Already stemmed!
Indexing...
Calculating tf-idf...
Best matching documents to 'pianists play piano':
Great Pianists on Piano Playing - James Francis Cooke: 9.924595e-02
Life of Chopin - Franz Liszt: 6.284168e-02
Violin Mastery_Talks with Master Violinists and Teachers - Frederick Herman Martens: 5.756910e-02
Resonance in Singing and Speaking - Thomas Fillebrown: 3.268530e-02
The Sorrows of Young Werther - Johann Wolfgang von Goethe: 2.782459e-02
Concerning the Spiritual in Art - Wassily Kandinsky: 2.308783e-02
The Phantom of the Opera - Gaston Leroux: 1.518331e-02
Crime and Punishment - Fyodor Dostoyevsky: 1.092031e-02
Around the World in Eighty Days - Jules Verne: 1.042157e-02
Anna Karenina - Leo Tolstoy: 9.780570e-03


![](http://www.quickmeme.com/img/5c/5c7dcf024101b083008e90529f42c1e32be6a97d47fc4c0c8f449466b9bc8613.jpg)
